In [47]:
import numpy as np

from sklearn.metrics import mean_squared_error

In [48]:
result_dir = '../results/'

# TablePrinter class

In [49]:
class TablePrinter:
    
    def __init__(self, metric_fns=[], header=None, precisions=[]):
        
        # Functions for metrics
        self.metric_fns = metric_fns
        self.n_metrics = len(self.metric_fns)
        self.precisions = precisions if len(precisions) == self.n_metrics else [2] * self.n_metrics
        
        # Total table and current block
        self.table = ''
        self.block_entries = []
        self.block_brackets = []

        # Formatting options
        self.indent = '   '
        self.col_sep = ' & '
        self.end_row = r'\\'
        self.midrule = r'\midrule'
        self.end_line = '\n'
        self.emphasis_begin = r'\mathbf{'
        self.emphasis_end = r'}'
        
        # Header
        self.table = ''
        if header is not None:
            self.table += self.indent + header + self.end_row + self.end_line
    
    
    def finalise_block(self):

        # Find best performance
        block_metrics = [line[2:] for line in self.block_entries]
        block_metrics = np.array(block_metrics)
        block_best = []
        for i in range(self.n_metrics):
            try:
                block_best.append(np.nanargmin(block_metrics[:,i]))
            except ValueError:
                block_best.append(-1)

        # Format entries
        text = ''
        for i, (line, brackets) in enumerate(zip(self.block_entries, self.block_brackets)):
            
            # Labels
            text += self.indent + line[0] + self.col_sep + line[1] + self.col_sep
            
            # Metrics
            for j in range(self.n_metrics):
                if np.isfinite(line[j + 2]):
                    if brackets[j+2]:
                        text += '(' + self.format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j])) + ')'
                    else:
                        text += self.format_number(line[j + 2], self.precisions[j], emphasize=(i == block_best[j]))
                if j == len(line) - 3:
                    text += self.end_row + self.end_line
                else:
                    text += self.col_sep

        # Add to document and reset for next block
        self.table += text
        self.block_entries = []
        self.block_brackets = []
    
    
    def new_block(self):
        self.finalise_block()
        self.table += self.indent + self.midrule + self.end_line
    
    
    def format_number(self,
                      number,
                      precision=2,
                      trailing_zeros=True,
                      fix_minus_zero=True,
                      latex_math_mode=True,
                      emphasize=False):
        if precision == 0:
            temp =  str(int(round(number,precision)))
        elif trailing_zeros:
            temp =  ('{:.' + str(precision) + 'f}').format(round(number,precision))
        else:
            temp =  str(round(number,precision))
        if fix_minus_zero and len(temp) > 0:
            if temp[0] == '-' and float(temp) == 0.:
                temp = temp[1:]
        if latex_math_mode:
            if emphasize:
                temp = '$\mathbf{' + temp + '}$'
            else:
                temp = '$' + temp + '$'
        elif emphasize:
            temp = r'\emph{' + temp + r'}'
        return temp
    
    
    def add(self, col1, col2, filename, folder='parameterized'):
        
        # Label columns
        line = [col1, col2]
        brackets = [False, False]
        
        # Metrics
        for fn in self.metric_fns:
            try:
                value = fn(filename, folder)
                bracket = False
                if value is dict:
                    value, bracket = value
            except IOError:
                #print('File', filename, 'in folder', folder, 'not found')
                value, bracket = np.nan, False
            line.append(value)
            brackets.append(bracket)

        self.block_entries.append(line)
        self.block_brackets.append(brackets)
    
    
    def print(self):
        self.finalise_block()
        return self.table

# Metrics

In [50]:
def mse_log_r_nottrained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_nottrained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_nottrained_' + filename + '.npy'))
    try:
        return mean_squared_error(log_r_truth, log_r_estimated)
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return mean_squared_error(log_r_truth[finites], log_r_estimated[finites]), True

def mse_log_r_trained(filename, folder='parameterized'):
    log_r_truth = np.log(np.load(result_dir + 'truth/r_trained_truth.npy'))
    log_r_estimated = np.log(np.load(result_dir + folder + '/r_trained_' + filename + '.npy'))
    try:
        return mean_squared_error(log_r_truth, log_r_estimated)
    except ValueError:
        finites = np.isfinite(log_r_truth) & np.isfinite(log_r_estimated)
        return mean_squared_error(log_r_truth[finites], log_r_estimated[finites]), True

def mse_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    try:
        return mean_squared_error(expected_log_r_truth, expected_log_r_estimated)
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites]), True

def mse_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    try:
        return mean_squared_error(expected_log_r_truth, expected_log_r_estimated)
    except ValueError:
        finites = np.isfinite(expected_log_r_truth) & np.isfinite(expected_log_r_estimated)
        return mean_squared_error(expected_log_r_truth[finites], expected_log_r_estimated[finites]), True

def mse_score_nottrained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_nottrained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_nottrained_' + filename + '.npy')
    return mean_squared_error(t_truth, t_estimated)

def mse_score_trained(filename, folder='parameterized'):
    t_truth = np.load(result_dir  + 'truth/scores_trained_truth.npy')
    t_estimated = np.load(result_dir + folder + '/scores_trained_' + filename + '.npy')
    return mean_squared_error(t_truth, t_estimated)

def var_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    return np.var(expected_log_r_truth - expected_log_r_estimated)

def var_delta_expected_log_r(filename, folder='parameterized'):
    expected_log_r_truth = np.load(result_dir + 'truth/llr_truth.npy')
    expected_log_r_truth -= np.min(expected_log_r_truth)
    expected_log_r_estimated = np.load(result_dir + folder + '/llr_' + filename + '.npy')
    expected_log_r_estimated -= np.min(expected_log_r_estimated)
    return np.var(expected_log_r_truth - expected_log_r_estimated)

# Tables

In [51]:
labels = ['PbP carl (raw)',
          'PbP carl (cal.)',
          'Param carl (raw)',
          'Param carl (cal.)',
          'Param score (raw)',
          'Param score (cal.)',
          'Param carl + score (raw)',
          'Param carl + score (cal.)',
          'PbP regression (raw)',
          'Param regression (raw)',
          'Param regr.\ + score (raw)']

point_by_point = [True, True,
                 False, False, False, False, False, False,
                 True,
                 False, False]

filenames = ['carl',
             'carl_calibrated',
             'carl',
             'carl_calibrated',
             'score',
             'score_calibrated',
             'combined',
             'combined_calibrated',
             'regression',
             'regression',
             'combinedregression']

In [52]:
table1 = TablePrinter([mse_log_r_trained, mse_log_r_nottrained,
                       mse_expected_log_r, mse_delta_expected_log_r,
                       var_expected_log_r, var_delta_expected_log_r],
                      precisions=[3,3,2,2,2,2])

for i, (label, filename, pbp) in enumerate(zip(labels[:4], filenames[:4], point_by_point[:4])):
    
    if i > 0:
        table1.new_block()
        
    if pbp:
        table1.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
        table1.add('', 'PbP (2)', filename, 'point_by_point')
        table1.add('', 'PbP (3)', filename + '_deep', 'point_by_point')
    
    else:
        table1.add(label, 'Baseline (1)', filename + '_shallow')
        table1.add('', 'Baseline (2)', filename)
        table1.add('', 'Baseline (3)', filename + '_deep')

        table1.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
        table1.add('', r'Random $\boldtheta$ (2)', filename + '_random')
        table1.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

        table1.add('', 'Physics-aware, baseline (1)', filename + '_aware_shallow')
        table1.add('', 'Physics-aware, baseline (2)', filename + '_aware')
        table1.add('', 'Physics-aware, baseline (3)', filename + '_aware_deep')

        table1.add('', 'Physics-aware, morphing basis (1)', filename + '_aware_basis_shallow')
        table1.add('', 'Physics-aware, morphing basis (2)', filename + '_aware_basis')
        table1.add('', 'Physics-aware, morphing basis (3)', filename + '_aware_basis_deep')

print(table1.print())

   PbP carl (raw) & PbP (1) &  &  &  &  &  & \\
    & PbP (2) & $\mathbf{0.149}$ & $\mathbf{0.141}$ &  &  &  & \\
    & PbP (3) &  &  &  &  &  & \\
   \midrule
   PbP carl (cal.) & PbP (1) &  &  &  &  &  & \\
    & PbP (2) & $\mathbf{0.029}$ & $\mathbf{0.035}$ &  &  &  & \\
    & PbP (3) &  &  &  &  &  & \\
   \midrule
   Param carl (raw) & Baseline (1) &  &  &  &  &  & \\
    & Baseline (2) & $\mathbf{0.037}$ & $\mathbf{0.044}$ & $5.45$ & $1.94$ & $\mathbf{1.34}$ & $\mathbf{1.34}$\\
    & Baseline (3) &  &  &  &  &  & \\
    & Random $\boldtheta$ (1) &  &  &  &  &  & \\
    & Random $\boldtheta$ (2) & $0.060$ & $0.075$ & $\mathbf{4.28}$ & $\mathbf{1.50}$ & $1.35$ & $1.35$\\
    & Random $\boldtheta$ (3) &  &  &  &  &  & \\
    & Physics-aware, baseline (1) &  &  &  &  &  & \\
    & Physics-aware, baseline (2) &  &  &  &  &  & \\
    & Physics-aware, baseline (3) &  &  &  &  &  & \\
    & Physics-aware, morphing basis (1) &  &  &  &  &  & \\
    & Physics-aware, morphing basis (2) &  &

In [53]:
table2 = TablePrinter([mse_log_r_trained, mse_log_r_nottrained,
                       mse_expected_log_r, mse_delta_expected_log_r,
                       var_expected_log_r, var_delta_expected_log_r],
                      precisions=[3,3,2,2,2,2])

for i, (label, filename, pbp) in enumerate(zip(labels[4:8], filenames[4:8], point_by_point[4:8])):
    
    if i > 0:
        table2.new_block()
        
    if pbp:
        table2.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
        table2.add('', 'PbP (2)', filename, 'point_by_point')
        table2.add('', 'PbP (3)', filename + '_deep', 'point_by_point')
        table2.add('', 'PbP learning log r (1)', filename + '_learnlogr_shallow', 'point_by_point')
        table2.add('', 'PbP learning log r (2)', filename + '_learnlogr', 'point_by_point')
        table2.add('', 'PbP learning log r (3)', filename + '_learnlogr_deep', 'point_by_point')
    
    else:
        table2.add(label, 'Baseline (1)', filename + '_shallow')
        table2.add('', 'Baseline (2)', filename)
        table2.add('', 'Baseline (3)', filename + '_deep')

        table2.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
        table2.add('', r'Random $\boldtheta$ (2)', filename + '_random')
        table2.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

        table2.add('', 'Physics-aware, baseline (1)', filename + '_aware_shallow')
        table2.add('', 'Physics-aware, baseline (2)', filename + '_aware')
        table2.add('', 'Physics-aware, baseline (3)', filename + '_aware_deep')

        table2.add('', 'Physics-aware, morphing basis (1)', filename + '_aware_basis_shallow')
        table2.add('', 'Physics-aware, morphing basis (2)', filename + '_aware_basis')
        table2.add('', 'Physics-aware, morphing basis (3)', filename + '_aware_basis_deep')

print(table2.print())

   Param score (raw) & Baseline (1) &  &  &  &  &  & \\
    & Baseline (2) & $\mathbf{0.057}$ & $\mathbf{0.057}$ & $\mathbf{0.67}$ & $\mathbf{0.14}$ & $\mathbf{0.09}$ & $\mathbf{0.09}$\\
    & Baseline (3) &  &  &  &  &  & \\
    & Random $\boldtheta$ (1) &  &  &  &  &  & \\
    & Random $\boldtheta$ (2) & $0.121$ & $0.160$ & $5.83$ & $3.16$ & $2.98$ & $2.98$\\
    & Random $\boldtheta$ (3) &  &  &  &  &  & \\
    & Physics-aware, baseline (1) &  &  &  &  &  & \\
    & Physics-aware, baseline (2) &  &  &  &  &  & \\
    & Physics-aware, baseline (3) &  &  &  &  &  & \\
    & Physics-aware, morphing basis (1) &  &  &  &  &  & \\
    & Physics-aware, morphing basis (2) &  &  &  &  &  & \\
    & Physics-aware, morphing basis (3) &  &  &  &  &  & \\
   \midrule
   Param score (cal.) & Baseline (1) &  &  &  &  &  & \\
    & Baseline (2) & $\mathbf{0.058}$ & $\mathbf{0.041}$ &  &  &  & \\
    & Baseline (3) &  &  &  &  &  & \\
    & Random $\boldtheta$ (1) &  &  &  &  &  & \\
    & Random $\

In [54]:
table3 = TablePrinter([mse_log_r_trained, mse_log_r_nottrained,
                       mse_expected_log_r, mse_delta_expected_log_r,
                       var_expected_log_r, var_delta_expected_log_r],
                      precisions=[3,3,2,2,2,2])

for i, (label, filename, pbp) in enumerate(zip(labels[8:], filenames[8:], point_by_point[8:])):
    
    if i > 0:
        table3.new_block()
        
    if pbp:
        table3.add(label, 'PbP (1)', filename + '_shallow', 'point_by_point')
        table3.add('', 'PbP (2)', filename, 'point_by_point')
        table3.add('', 'PbP (3)', filename + '_deep', 'point_by_point')
    
    else:
        table3.add(label, 'Baseline (1)', filename + '_shallow')
        table3.add('', 'Baseline (2)', filename)
        table3.add('', 'Baseline (3)', filename + '_deep')

        table3.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
        table3.add('', r'Random $\boldtheta$ (2)', filename + '_random')
        table3.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

        table3.add('', 'Physics-aware, baseline (1)', filename + '_aware_shallow')
        table3.add('', 'Physics-aware, baseline (2)', filename + '_aware')
        table3.add('', 'Physics-aware, baseline (3)', filename + '_aware_deep')

        table3.add('', 'Physics-aware, morphing basis (1)', filename + '_aware_basis_shallow')
        table3.add('', 'Physics-aware, morphing basis (2)', filename + '_aware_basis')
        table3.add('', 'Physics-aware, morphing basis (3)', filename + '_aware_basis_deep')

print(table3.print())

   PbP regression (raw) & PbP (1) &  &  &  &  &  & \\
    & PbP (2) & $\mathbf{0.003}$ & $\mathbf{0.003}$ &  &  &  & \\
    & PbP (3) &  &  &  &  &  & \\
   \midrule
   Param regression (raw) & Baseline (1) &  &  &  &  &  & \\
    & Baseline (2) & $\mathbf{0.004}$ & $\mathbf{0.005}$ & $\mathbf{0.07}$ & $0.07$ & $0.07$ & $0.07$\\
    & Baseline (3) &  &  &  &  &  & \\
    & Random $\boldtheta$ (1) &  &  &  &  &  & \\
    & Random $\boldtheta$ (2) & $0.005$ & $0.006$ & $0.08$ & $\mathbf{0.04}$ & $\mathbf{0.04}$ & $\mathbf{0.04}$\\
    & Random $\boldtheta$ (3) &  &  &  &  &  & \\
    & Physics-aware, baseline (1) &  &  &  &  &  & \\
    & Physics-aware, baseline (2) &  &  &  &  &  & \\
    & Physics-aware, baseline (3) &  &  &  &  &  & \\
    & Physics-aware, morphing basis (1) &  &  &  &  &  & \\
    & Physics-aware, morphing basis (2) & $0.015$ & $85.603$ & $9572.27$ & $18995.38$ & $7907.55$ & $7907.55$\\
    & Physics-aware, morphing basis (3) &  &  &  &  &  & \\
   \midrule
   Param 

In [55]:
labels = ['Param carl',
          'Param score',
          'Param carl + score',
          'Param regression',
          'Param regr.\ + score']

filenames = ['carl',
             'score',
             'combined',
             'regression',
             'combinedregression']

table4 = TablePrinter([mse_score_trained, mse_score_nottrained],
                      precisions=[2,2])

for i, (label, filename) in enumerate(zip(labels, filenames)):
    
    if i > 0:
        table4.new_block()
        
    table4.add(label, 'Baseline (1)', filename + '_shallow')
    table4.add('', 'Baseline (2)', filename)
    table4.add('', 'Baseline (3)', filename + '_deep')
    
    table4.add('', r'Random $\boldtheta$ (1)', filename + '_random_shallow')
    table4.add('', r'Random $\boldtheta$ (2)', filename + '_random')
    table4.add('', r'Random $\boldtheta$ (3)', filename + '_random_deep')

    table4.add('', 'Physics-aware, baseline (1)', filename + '_aware_shallow')
    table4.add('', 'Physics-aware, baseline (2)', filename + '_aware')
    table4.add('', 'Physics-aware, baseline (3)', filename + '_aware_deep')

    table4.add('', 'Physics-aware, morphing basis (1)', filename + '_aware_basis_shallow')
    table4.add('', 'Physics-aware, morphing basis (2)', filename + '_aware_basis')
    table4.add('', 'Physics-aware, morphing basis (3)', filename + '_aware_basis_deep')

print(table4.print())

   Param carl & Baseline (1) &  & \\
    & Baseline (2) & $\mathbf{0.81}$ & $\mathbf{1.21}$\\
    & Baseline (3) &  & \\
    & Random $\boldtheta$ (1) &  & \\
    & Random $\boldtheta$ (2) & $0.87$ & $1.23$\\
    & Random $\boldtheta$ (3) &  & \\
    & Physics-aware, baseline (1) &  & \\
    & Physics-aware, baseline (2) &  & \\
    & Physics-aware, baseline (3) &  & \\
    & Physics-aware, morphing basis (1) &  & \\
    & Physics-aware, morphing basis (2) &  & \\
    & Physics-aware, morphing basis (3) &  & \\
   \midrule
   Param score & Baseline (1) &  & \\
    & Baseline (2) & $\mathbf{0.10}$ & $\mathbf{0.24}$\\
    & Baseline (3) &  & \\
    & Random $\boldtheta$ (1) &  & \\
    & Random $\boldtheta$ (2) & $0.41$ & $0.77$\\
    & Random $\boldtheta$ (3) &  & \\
    & Physics-aware, baseline (1) &  & \\
    & Physics-aware, baseline (2) &  & \\
    & Physics-aware, baseline (3) &  & \\
    & Physics-aware, morphing basis (1) &  & \\
    & Physics-aware, morphing basis (2) &  & \\
 